In [32]:
# LAB 3: Prompt Engineering Patterns & Conversation Memory
# Google Colab + LangChain + Hugging Face
# STEP 1: Install required libraries
!pip install -q --upgrade langchain langchain-community transformers accelerate sentencepiece langchain-core

In [33]:
import requests
print(requests.__version__)

2.32.5


In [34]:
# STEP 2: Load Hugging Face LLM (FLAN-T5)
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=256
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu


In [35]:
# STEP 3: Import PromptTemplate and Memory components
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# Try importing memory - handle different LangChain versions
try:
    from langchain.memory import ConversationBufferMemory
    from langchain.chains import ConversationChain
except ImportError:
    try:
        from langchain_community.memory import ConversationBufferMemory
        from langchain.chains import ConversationChain
    except ImportError:
        print("Note: Using alternative memory approach")
        ConversationBufferMemory = None
        ConversationChain = None

Note: Using alternative memory approach


In [36]:
# STEP 4: Demonstration WITHOUT Memory (Problem)
simple_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the question:\n{question}"
)

print("=== WITHOUT MEMORY ===")
print("Q: Who is Steve Jobs?")
print("A:", llm.invoke(simple_prompt.format(
    question="Who is Steve Jobs?"
)))

print("\nQ: What is his wife name?")
print("A:", llm.invoke(simple_prompt.format(
    question="What is his wife name?"
)))

=== WITHOUT MEMORY ===
Q: Who is Steve Jobs?
A: computer programmer

Q: What is his wife name?
A: elizabeth harris


In [39]:
conversation_history = []

def ask_with_memory(question):
    """Ask a question with conversation memory"""
    # Build context from conversation history
    history_context = ""
    if conversation_history:
        history_context = "\n\nPrevious conversation:\n"
        for i, (q, a) in enumerate(conversation_history, 1):
            history_context += f"Q{i}: {q}\nA{i}: {a}\n"

    # Create prompt with history
    prompt_text = f"""You are a helpful assistant. Use the conversation history to answer questions.{history_context}

Current question: {question}

Answer:"""

    # Get response
    response = llm.invoke(prompt_text)

    # Store in memory
    conversation_history.append((question, response))

    return response

# Clear previous conversation
conversation_history = []

print("=== WITH MEMORY ===")
print("Q: Who is Steve Jobs?")
response1 = ask_with_memory("Who is Steve Jobs?")
print("A:", response1)

print("\nQ: What is his wife name?")
response2 = ask_with_memory("What is his wife name?")
print("A:", response2)

print("\nQ: What company did he co-found?")
response3 = ask_with_memory("What company did he co-found?")
print("A:", response3)


=== WITH MEMORY ===
Q: Who is Steve Jobs?
A: Steve Jobs

Q: What is his wife name?
A: Sarah Michelle Gellar

Q: What company did he co-found?
A: Apple Inc.
